In [ ]:
import shutil

from astrocast.analysis import IO, Video, Events

from pathlib import Path

Here, we will showcase how to run the astrocast pipeline on a small sample dataset. 

# 0. Explore the sample data

In [ ]:
sample_path = Path("../testdata/sample_2.h5")
assert sample_path.exists()

# we can use the visualize_h5 function from the command line interface (CLI)
!astrocast visualize-h5 {sample_path.as_posix()}
print("\n")

In [ ]:
# next we load the dataset
io = IO()
data = io.load(sample_path, h5_loc="data/ch0")
print(f"Data shape: {data.shape}, dtype: {data.dtype}, instance type: {type(data)}")

# and visualize the data
video = Video(data)
fig = video.plot_overview()

# uncomment to activate the video player
# viewer = video.show()  

# 1. Preprocessing

In [ ]:
from astrocast.preparation import MotionCorrection, Input, Delta
from astrocast.denoising import SubFrameGenerator


## 1.1 Data conversion

In [ ]:
test_path = Path("./quickstart.h5")

In [ ]:
# multiple ways exist to convert the data to a format that is compatible with the pipeline
# including .tiff (multi and single page), .avi, .h5 and .czi files
# here we do a simple 'copy' of the sample file

if test_path.exists():
    test_path.unlink()
assert not test_path.exists()

inp = Input()
inp.run(input_path=sample_path, output_path=test_path, h5_loc_in="data/ch0", h5_loc_out="data/ch0", compression="gzip", infer_strategy="Z")

# alternatively with CLI:
# !astrocast convert-h5 --output-path {test_path.as_posix()} --h5-loc-in data/ch0 --h5-loc-out data/ch0 --compression gzip --infer-chunks {sample_path.as_posix()}

# 1.2 Motion correction

In [ ]:
MC = MotionCorrection()
MC.run(input_=test_path, h5_loc="data/ch0")
MC.save(output=test_path, h5_loc="mc/ch0", infer_strategy="Z", compression="gzip")

In [ ]:
!astrocast visualize-h5 {test_path.as_posix()}

In [ ]:
# TODO some kind of plotting to show the motion correction!?

## 1.3 Denoising

In [ ]:
# Download pretrained models
# !astrocast download-models "./models"--

In [ ]:
# data_generator = SubFrameGenerator(paths=test_path, input_size=(64, 64), pre_post_frame=5, gap_frames=0, batch_size=64, loc="mc/ch0",
#                                    overlap=5, padding="edge",normalize="global")
# 
# denoising_model = Path("./models/1p_exvivo_input_size_64_64-pre_post_frame_5-gap_frames_0-train_rotation_1_2_3-architecture_3_64.h5")
# assert denoising_model.exists()
# 
# data_generator.infer(model=denoising_model, output=test_path, out_loc="dn/ch0")

In [ ]:
# !astrocast visualize-h5 {test_path.as_posix()}

# 1.4 Subtract background

In [ ]:
delta = Delta(test_path, loc="mc/ch0")
delta.run(window=10, method="dF")
delta.save(output_path=test_path, h5_loc="df/ch0", infer_strategy="balanced", compression="gzip")

# TODO add terminal command

In [ ]:
!astrocast visualize-h5 {test_path.as_posix()}

In [ ]:
# TODO visualize results

# 2. Event detection

In [ ]:
from astrocast.detection import Detector
import shutil

In [ ]:
event_path = test_path.with_suffix(".ch0.roi")
if event_path.exists():
    shutil.rmtree(event_path)

detector = Detector(input_path=test_path)
event_dictionary = detector.run(h5_loc="df/ch0", debug=True, exclude_border=20, temporal_prominence=0.3)

In [ ]:
assert event_path.exists()
!ls -lthv {event_path.as_posix()}

events = Events(event_path)
events

In [ ]:
#!astrocast view-detection-results --video-path {test_path.as_posix()} --h5-loc df/ch0 {event_path.as_posix()}